# MONTECARLO PARALELIZADO

## Instalaciones necesarias

- Para que el código a continuación funcione, es necesario tener instalado `dask`.

In [ ]:
%pip install --user dask   

In [ ]:
%pip install --user dask distributed

- Para habilitar el dashboard, debemos installar `bokeh` :

In [ ]:
%pip install --user bokeh

In [ ]:
%%bash 
sudo apt-get install -y graphviz

In [ ]:
%pip install --user -q graphviz

In [ ]:
%pip install --user numpy 

In [2]:
import IPython
app=IPython.Application.instance()
app.kernel.do_shutdown(True) #restarting kernell

{'status': 'ok', 'restart': True}

In [1]:
from IPython.display import Image
import time

## Montecarlo 1D


Empezamos con el algoritmo de monte carlo en 1 dimensión

Integramos la función:
$$f(x)=\int_0^1 \frac{4}{1+x^2}=\pi$$ 




### Método secuencial:

In [2]:
import numpy as np
import math

In [3]:
# definimos la funcion del error relativo
err_rel=lambda ap,ob: math.fabs(ap-ob)/math.fabs(ob)

In [4]:
# número de puntos a utilizar
density_p=10**7

# función a integrar
f=lambda x: 4/(1+x**2)

# resultado conocido de la integral (valor objetivo)
obj=math.pi

# intervalo
a=0
b=1
# dimensiones
dims=1

In [5]:
%%time
#approximación mediante método montecarlo
x_p = np.random.uniform(0,1,density_p) #obtiene tantos números aleatorios entre 0 y 1 como density_p.
vol = b-a # esta es la longitud del intervalo de integración.
approx = vol*np.mean(f(x_p))

print("---------------------")
print("El valor calculado de la integral es: ",approx)
print("El error relativo: {:0.4e}".format(err_rel(approx,obj)))
print("---------------------")

---------------------
El valor calculado de la integral es:  3.141410099666798
El error relativo: 5.8109e-05
---------------------
CPU times: user 167 ms, sys: 70.2 ms, total: 237 ms
Wall time: 242 ms


A continuación, simplemente se vuelve función el proceso anterior para poder evaluar de manera eficiente varias veces el computo de dicha función. 

In [6]:
def montecarlo_1D(density_p, f, a=0, b=1):
    '''
    Esta función aproxima la integral en el intervalo [a,b] por el método de montecarlo en 1 dimensión para la función
    f(x).
    _______
    Argumentos:
        - density_p (float): Densisdad para determinar cuantos puntos aleatorios se generan en cada dimensión a partir de una
        distribución uniforme.
        - f (lambda): función sobre la cual se va a integrar
        - a: punto inferior del intervalo sobre el que se va a integrar
        - b: punto inferisuperior del intervalo sobre el que se va a integrar
    Salidas:
        - aprox: aproximación de la integral por Montecarlo
    '''
    x_p = np.random.uniform(0,1,density_p) 
    vol = b-a
    aprox = vol*np.mean(f(x_p))
    
    return aprox

In [7]:
start_time = time.time()
aprox = montecarlo_1D(density_p=10**7, f=lambda x: 4/(1+x**2), a=0, b=1)
end_time = time.time()
secs = end_time-start_time

# lambda statement para calcualr el error relativo
err_rel=lambda ap,ob: math.fabs(ap-ob)/math.fabs(ob)
# valor de la integral
obj=math.pi

print("Evalución de integral de montecarlo de forma secuencial:")
print("---------------------")
print("El valor calculado de la integral es: ",aprox)
print("El error relativo: {:0.4e}".format(err_rel(aprox,obj)))
print(f"El tiempo de ejecución: {secs} segundos")
print("---------------------")

Evalución de integral de montecarlo de forma secuencial:
---------------------
El valor calculado de la integral es:  3.1417272402444905
El error relativo: 4.2840e-05
El tiempo de ejecución: 0.30786824226379395 segundos
---------------------


Utilizando `time`, el tiempo de ejecución gue de 3.14 segundos. 

Ahora se evaluará de nuevo el algoritmo, utilizando el `magic command`, `timeit`, el cual permite calcular el tiempo de ejecución de un programa en python tras ser ejecutado en un ciclo por `-n` veces, repitiendo este procedimiento `-r` veces. El flag `-o` se utiliza para tener una salida de tipo timeit que pueda ser almacenada en una variable, en este caso, "montecarlo_secuencial_timeit".

In [8]:
montecarlo_secuencial_timeit = %timeit -n 5 -r 10 -o montecarlo_1D(density_p=10**7, f=lambda x: 4/(1+x**2), a=0, b=1)

151 ms ± 2.31 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


Los anteriores resultados en términos de tiempo son consistentes a los ejercicios realizados incialmente para le evaluación de desempeño del programa generado secuencialmnte.

### Paralelización de MC en Dask

Para paralelizar estos resultados lo que pienso que tenemos que hacer es dividir las tareas en n subintervalos. La porpuesta es la siguiente:  
1. Dividir el dominio de la función en $p$ partes iguales. $p$ será el número de cpus disponibles en la máquina en que se corre la tarea.
2. Dividir el total de puntos a utilizar equitativamente entre $p$.
3. Resolver el método para cada intervalo.
4. Sumar los resultados de cada intervalo.

**Nota:** Para el caso de más dimensiones solamente se paraleliza una dimensión pues al hacerlo con todas se estaría dividiendo el espacio en $p_{dim}$ subespacios.

In [11]:
import multiprocessing
import time
from dask.distributed import Client
client = Client()

In [12]:
client

Client Scheduler: tcp://127.0.0.1:62080 Dashboard: http://127.0.0.1:62081/status,Cluster Workers: 4 Cores: 8 Memory: 8.59 GB


Si se quiere ser más explícito con el uso de los recursos computacionales:

In [13]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='6GB')
client

Client Scheduler: inproc://192.168.0.27/17241/8 Dashboard: http://192.168.0.27:62102/status,Cluster Workers: 1 Cores: 4 Memory: 6.00 GB


In [14]:
# paso 1. Dividir el dominio en partes iguales
p = multiprocessing.cpu_count() # cpus disponibles
n_subint = int(density_p/p) # número de puntos o nodos en cada core o cpu

In [15]:
def construye_subintervalos1D(ids,a,b,p,dims):
    """
    Función que construye "p" subintervalos, delimitados entre a y b. 
    El número de subintervalos dependerá del número de cores empleados. 
    Este ejercicio fue ejecutado en una computadoracon 16 cores, por lo 
    tanto, se crean 16 subintervalos.
    
    Argumentos:
    ----------
    * ids: Identificador del core dónde se está corriendo el task. 
    Este ejercicio fue ejecutado en una computadora con 16 cores, por 
    ello los posibles valores de "ids" son números enteros entre 0 y 15.
    * a, b (float): Intervalo de integración [a, b]
    * p (int) : Número de cores o cpus disponibles
    * dims: Dimensión de la integral.
    
    Salidas:
    -------
    * (begin, end): Subintervalos de integración.
    
    Ejemplo:
    -------
    construye_subintervalos(ids=0,a,b,p,dims)
    return:
    (0.0, 0.0625)

    """
    tamano_int = (b-a)/p #tamaño de cada sub intervalo.
    begin = a + ids*tamano_int #construyen los subintervalo
    end = begin+tamano_int
    return (begin,end)

    
def evalua_subintervalos1D(intervalo,f,n_subint,dims):
    """
    Función que evalúa la integral en cada uno los p subintervalos \\
    previamente construidos con la función "construye_subintervalos"
    
    Argumentos:
    ----------
    * intervalo: Intervalo de la integral sobre la cual se va aproximar \\
                 la integral
    * f: Función que se está aproximando
    * n_subint: Número de puntos o nodos en cada core o cpu
    * dims: Dimensiones de la integral
    
    Salidas:
    --------
    + vol*np.mean(f(x_p)): Approximación a la integral
    
    """   
    x_p=np.random.uniform(intervalo[0],intervalo[1],n_subint) #draw samples from a uniform distribution
    vol=intervalo[1]-intervalo[0]
    return vol*np.mean(f(x_p)) #calcula la integral para una cada regió: subintervalo.

In [16]:
%%time

# submitting p function calls
futures_intervalos = client.map(construye_subintervalos1D,range(p),
                                                **{'a':a,
                                                   'b':b,
                                                   'p':p,
                                                   'dims':dims})

futures_evaluando=client.map(evalua_subintervalos1D, futures_intervalos,
                                             **{'f':f,
                                                'n_subint':n_subint,
                                                'dims':dims})

results=client.gather(futures_evaluando) # los resultados viven en varios workers 
aprox=sum(results) #sumamos los resultados de todos los workers

print("---------------------")
print("El valor calculado de la integral es: ",aprox)
print("El error relativo: {:0.4e}".format(err_rel(aprox,obj)))
print("---------------------")

---------------------
El valor calculado de la integral es:  3.1415475198108935
El error relativo: 1.4367e-05
---------------------
CPU times: user 238 ms, sys: 79.1 ms, total: 317 ms
Wall time: 305 ms


- Utilizando una computadora de 16 núcleos (y por lo tanto 16 intervalos)con 8 GB de ram:
Para el método en secuencial obtenemos un error relativo de $3.6251x10^{-6}$ y un tiempo de ejecución de $2.38$ seg y para el método en paralelo obtenemos un error relativo de $6.0184x10^{-7}$ en $110$ ms.

- Utilizando una computadora de 8 núcleos (y por lo tanto 8 intervalos)con 4 GB de ram:
se obtiene un error relativo de $1.8433x10^{-7}$ y se tarda un tiempo de ejecución de 428 ms.

Con el fin de visualizar el tiempo de ejecución de las tareas paralelizadas, se presenta a continuación la gráfica generada en `dask` por medio de `bokeh`:

![dask_tasKstream](imgs/mc_1dim_dask_taskStream.png)

A pesar de que los colores sean casi indistinguibles entre las dos tareas, es posible diferenciarlas por medio de la magnitud de sus barras. El bloque "construye_subintervalos" es la tarea más rápida,dado que solo implica la construcción de subintervalos en función del número de cores empledos en la tarea. La tarea "evalua_subintervalos" es la que mayor tiempo de computo lleva, en la medida que implica el cálculo de un área para cada subintervalor, lo que implica mayor complejidad. 

**Comparación de algunos resultados de computaciones**

In [17]:
import dask
%time dask.compute(*results)

CPU times: user 1.92 ms, sys: 3.12 ms, total: 5.04 ms
Wall time: 3.89 ms


(0.49741702207072097,
 0.48250344253961425,
 0.4551670640739827,
 0.419499425930118,
 0.37978895076268315,
 0.33959927856281175,
 0.3013085348700217,
 0.266263801000941)

In [18]:
futures = dask.persist(*results)
futures

(0.49741702207072097,
 0.48250344253961425,
 0.4551670640739827,
 0.419499425930118,
 0.37978895076268315,
 0.33959927856281175,
 0.3013085348700217,
 0.266263801000941)

In [19]:
client.cluster.scale(10)

In [20]:
client.close()

# Monte carlo en n-dimensiones

Se hace la generalización en que solamente se divide la primera dimensión y se paraleliza sobre esa dimensión usando las otras diemnsiones completas.  
Los intervalos sobre los que se trabaja son intervalos divididos en $p$ partes iguales para la primera dimensión y el intervalo de intergración completo para cada una de las siguientes dimensiones.

In [46]:
#funciones para multiples dimensiones

def construye_subintervalos(ids,a,b,p,dims):
    """
    Función que construye "p" subintervalos, delimitados entre a y b para la primera dimensión.    
    En las demás dimensiones integra sobre el intervalo completo.
    
    Argumentos:
    ----------
    * ids: Identificador del core dónde se está corriendo el task. 
    * a (float): Vector con el límite inferior de cada intervalo de integración [ax1,ax2,...,axn] para cada diemnsión. Tiene dims entradas.
    * b (float): Vector con el límite superior de cada intervalo de integración [bx1,bx2,...,bxn] para cada diemnsión. Tiene dims entradas.
    * p (int) : Número de cores o cpus disponibles
    * dims: Dimensiónes de la integral.
    
    Salidas:
    -------
    * intervalos : lista de pares de intervalos de integración ordenados por dimensión
    
    """
    if(dims==1):
        tamano_int=(b-a)/p
        begin= a + ids*tamano_int
        end=begin+tamano_int
        return (begin,end)

    else:
        tamano_int=(b-a)/p

        #aux solamente cambia la primera dimension
        aux=np.identity(dims)
        aux=aux[0,:]
        aux=aux*ids*tamano_int
        begin= a + aux
        
        aux=np.identity(dims)
        aux=aux[0,:]
        aux=aux*tamano_int
        end=begin+aux
        
        intervalos=[]
        for i in range(dims):
            if(i==0):
                intervalos=np.append(intervalos,[begin[i],end[i]])
            else:
                intervalos=np.append(intervalos,[a[i],b[i]])
                
        return intervalos

In [47]:
def evalua_subintervalos(intervalo,f,n_subint,dims):
    """
    Función que evalúa la integral en cada uno los p subintervalos \\
    previamente construidos con la función "construye_subintervalos"
    
    Argumentos:
    ----------
    * intervalo: Intervalo de la integral sobre la cual se va aproximar \\
                 la integral
    * f: Función que se está aproximando
    * n_subint: Número de puntos o nodos en cada core o cpu
    * dims: Dimensiones de la integral
    
    Salidas:
    --------
    + vol*np.mean(f(x)): Approximación a la integral
    
    """   

    x=[]
    vol=1
    for i in range(dims):
        x=np.append(x,np.random.uniform(intervalo[2*i],intervalo[2*i+1],n_subint))
        vol=vol*(intervalo[2*i+1]-intervalo[2*i])
    
    x=np.reshape(x,(n_subint,dims), order='F')
    return np.mean(f(x))*vol

### Ejemplo 1
 $ \int_0^2 \int_{-1}^1 \int_0^1 (2x+3y+z)dzdydx = 10$
 
metodo en paralelo

Primero se realiza una aplicación del método sin utilizar computo en paralelo, para efectos de comparación.
Sin embargo, cabe aclarar que la función `aprox_montecarlo` no es directamente comparable en la medida que 

In [48]:
def aprox_montecarlo(density_p, f, dims, x0=0, x1=2, y0=-1, y1=1, z0=0, z1=1):
    '''
    Función que computa aproximación de montecarlo ghasta en 3 dimensiones para la función n-variada en los intervalos
    definidos.
    _______
    Argumentos:
        - density_p (float): Densisdad para determinar cuantos puntos aleatorios se generan en cada dimensión a partir de una
        distribución uniforme.
        - f (lambda): función sobre la cual se va a integrar
        - dims (int): número de dimensiones sobre las que se va a integrar
    Salidas:
        - aprox: aproximación de la integral por Montecarlo
    '''
    if dims == 3:
        x_p=np.random.uniform(x0,x1,density_p)
        y_p=np.random.uniform(y0,y1,density_p)
        z_p=np.random.uniform(z0,z1,density_p)
        
        vol=(x1-x0)*(y1-y0)*(z1-z0)
        approx=vol*np.mean(f(x_p,y_p,z_p))
        
    elif dims == 2:
        x_p=np.random.uniform(x0,x1,density_p)
        y_p=np.random.uniform(y0,y1,density_p)
        
        vol=(x1-x0)*(y1-y0)*(z1-z0)
        approx=vol*np.mean(f(x_p,y_p,z_p)) 
        
    return aprox

Ahora evaluamos el rendimiento de esta función en términos de tiempo.

In [49]:
start_time = time.time()
aprox = aprox_montecarlo(density_p=10**7, f=lambda x,y,z: 2*x+3*y+z, dims=3)
end_time = time.time()
secs = end_time-start_time

# lambda statement para calcualr el error relativo
err_rel=lambda ap,ob: math.fabs(ap-ob)/math.fabs(ob)

print("Evalución de integral de montecarlo de forma secuencial:")
print("---------------------")
print("El valor calculado de la integral es: ",aprox)
print("El error relativo: {:0.4e}".format(err_rel(aprox,10)))
print(f"El tiempo de ejecución: {secs} segundos")
print("---------------------")

Evalución de integral de montecarlo de forma secuencial:
---------------------
El valor calculado de la integral es:  3.1415475198108935
El error relativo: 6.8585e-01
El tiempo de ejecución: 0.7828741073608398 segundos
---------------------


evaluación utilizando `%%time`

In [50]:
%%time
aprox = aprox_montecarlo(density_p=10**7, f=lambda x,y,z: 2*x+3*y+z, dims=3)
print("Evaluación de integral de montacarlo de forma secuencial:")
print("---------------------")
print("El valor calculado de la integral es: ",aprox)
print("El error relativo: {:0.4e}".format(err_rel(aprox,10)))
print(f"El tiempo de ejecución: {secs} segundos")
print("* Intervalo par el área: ")
print((approx-vol*error_est, approx+vol*error_est))
print("---------------------")

Evaluación de integral de montacarlo de forma secuencial:
---------------------
El valor calculado de la integral es:  3.1415475198108935
El error relativo: 6.8585e-01
El tiempo de ejecución: 0.7828741073608398 segundos
* Intervalo par el área: 
(-21.15199619974249, 41.14789456957055)
---------------------
CPU times: user 422 ms, sys: 59.9 ms, total: 482 ms
Wall time: 415 ms


Como se observa anteriormente, el tiempo de usuario de CPU, _CPU times_, (el tiempo que el procesador trabajó en el programa ejectuado) fue de 455 ms, mientras que el tiempo de _sys_ (lo que el procesador tarda en conectarse a funciones necesarias para que el programa se ejecute) fue de 57.8 ms. Por último, el _wall time_ calculado (tiempo que tarda el programa en ejecutarse de inicio a fin), fue de 513 ms. 

__pregunta para erick:__ ¿por qué es posible que wall time sea menor a total time (sys + CPU?

Ahora se evaluará de nuevo el algoritmo, utilizando el `magic command`, `timeit`, el cual permite calcular el tiempo de ejecución de un programa en `python` tras ser ejecutado en un ciclo por `-n` veces, repitiendo este procedimiento `-r` veces. El _flag_ `-o` se utiliza para tener una salida de tipo `timeit` que pueda ser almacenada en una variable, en este caso, "montecarlo_secuencial_timeit".

In [182]:
montecarlo_secuencial_timeit = %timeit -n 5 -r 10 -o aprox_montecarlo(density_p=10**7, f=lambda x,y,z: 2*x+3*y+z, dims=3)

382 ms ± 23 ms per loop (mean ± std. dev. of 10 runs, 5 loops each)


Luego de realizar el ejercicio anterior para obtener el rendimiento promedio del computo de nuestro programa, se obtiene que este se tarda aproximadamente 382 ms por loop, con una desviación de 23 ms. El resultado de realizar el ejercicio varias veces es consistente con el reportado anteriormente.

Ahora, se ejectua el ejercicio utilizando su **implementación en paralelo**:

In [51]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:62231 Dashboard: http://127.0.0.1:62232/status,Cluster Workers: 4 Cores: 8 Memory: 8.59 GB


In [39]:
# numero de procesadores
p = multiprocessing.cpu_count()
print(f"Número de procesadores: {p}")

Número de procesadores: 8


In [52]:
# numero de puntos a utlizar
density_p = 10**7
n_subint = int(density_p/p)

# funcion a integrar
# cada variable se toma como la columna de una matriz
f = lambda x: 2*x[:,0]+3*x[:,1]+x[:,2]

#reultado conocido de la integral (valor objetivo)
obj=10

# intervalo
# se debe de poner (x1_a,x2_a,...,xn_a) para los mínimos
a = np.array([0,-1,0])
# se debe de poner (x1_b,x2_b,...,xn_b) para los máxios
b = np.array([2,1,1])

# volumen
vol = np.prod(b-a)

#dimensiones
dims=len(a)

In [53]:
%%time
#llamado para multiples dimensiones
futures_intervalos = client.map(construye_subintervalos, range(p),
                                            **{'a':a,
                                               'b':b,
                                               'p':p,
                                               'dims':dims})

futures_evaluando=client.map(evalua_subintervalos, futures_intervalos,
                                            **{'f':f,
                                               'n_subint':n_subint,
                                               'dims':dims})

results=client.gather(futures_evaluando)
aprox=sum(results)

print("---------------------")
print("El valor calculado de la integral es: ",aprox)
print("El error relativo: {:0.4e}".format(err_rel(aprox,obj)))
print("---------------------")

---------------------
El valor calculado de la integral es:  9.995916280438724
El error relativo: 4.0837e-04
---------------------
CPU times: user 179 ms, sys: 52.5 ms, total: 232 ms
Wall time: 541 ms


Como se puede observar, en este primer ejercicio de medición, el tiempo tanto en _CPU times_, como en _sys_ y _total_
es menor en la implementación en paralelo. El resultado también es consistente con las mediciones del tiempo de inicia fin de ejecución del programa entero, correspondiente a _walltime_.

Con el fin de visualizar el tiempo de ejecución de las tareas paralelizadas, se presenta a continuación la gráfica generada en `dask` por medio de `bokeh`:

![dask_tasKstream](imgs/mc_ndim_dask_taskStream.png)

Como se puede observar, al igual que en ejercicio de montecarlo para una dimensión, el bloque `construye_subintervalos` es la tarea que menos tiempo tarda en ejecutarse, lo cual podría ser anticipable, en la medida que solo implica la construcción de subintervalos en función del número de _cores_ empledos en la tarea. Por otro lado, la tarea `evalua_subintervalos` es la que mayor tiempo de computo lleva, en la medida que implica el cálculo de un área para cada subintervalor, lo que implica mayor complejidad.  

In [33]:
client.close()

# Referencias:
- Notas de clase de MNO 2020, disponibles en: https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.2.Python_dask.ipynb
- Documentación de Dask, disponible [aquí](https://docs.dask.org/en/latest/)